## Flujos y pasos de la configuración con IaaC

IaaC es una tecnología que usamos en este proyecto para un manejo más controlado de este. En este caso, usamos __Terraform__ como herramienta para levantar el backend. En el proyecto, usamos 6 archivos distintos para el uso de este.

### Inicialización de Terraform

Partimos con el siguiente código en la terminal para introducirlo:

In [ ]:
terraform init
terraform validate

Agrega los módulos y validate es una buena práctica para el manejo de errores. Con estos comandos, se crea la carpeta 'terraform' con archivos incluidos como 'terraform.lock.hcl'.

### Archivos

1. __appsec.yml__

In [ ]:
version: 0.0
os: linux
files:
  # Que archivos se copiaran en la instancia
  # https://docs.aws.amazon.com/codedeploy/latest/userguide/tutorials-on-premises-instance-2-create-sample-revision.html
  - source: ./docker-compose.production.yml
    # Carpeta donde lo vamos a dejar
    destination: /home/ubuntu/
  - source: scripts
    destination: /home/ubuntu/scripts
  - source: .env
    destination: /home/ubuntu/

# Ciclo de vida del deployment 
# https://docs.aws.amazon.com/codedeploy/latest/userguide/reference-appspec-file-structure-hooks.html
hooks:
  ApllicationStop:
    - location: ./scripts/application-stop.sh
      timeout: 10000
  AfterInstall:
    - location: ./scripts/after-install.sh
      timeout: 3000
  ApplicationStart:
    - location: ./scripts/application-start.sh
      timeout: 3000
  ValidateService:
    - location: ./scripts/validate-service.sh
      timeout: 3000

En este código CodeDeploy copia archivos del repositorio a las rutas de la instancia, además inicializar scripts en orden específicos usando hooks y se espera a que se completen todas las validaciones

2. __deploy.sh__

In [ ]:
S3_BUCKET="iic2173-back-terraform"
ZIP_FILE="deploy.zip"
APPLICATION_NAME="iic2173-app-terraform"
DEPLOYMENT_GROUP_NAME="group-iic2173-terraform"

zip -r $ZIP_FILE ../scripts/ ./appspec.yml ../docker-compose.production.yml ../.env

aws s3 cp $ZIP_FILE s3://$S3_BUCKET/$ZIP_FILE

aws deploy create-deployment \
  --application-name $APPLICATION_NAME \
  --deployment-group-name $DEPLOYMENT_GROUP_NAME \
  --region us-east-1 \
  --s3-location bucket=$S3_BUCKET,key=$ZIP_FILE,bundleType=zip \
  --file-exists-behavior OVERWRITE

Este este código tiene funciones como crear un archivo zip con los recursos necesarios para inicializar el back (S3_BUCKET, APPLICATION_NAME, etc), sube el archivo zip al bucket S3 y se despliega en AWS CodeDeploy

3. __main.tf__

In [ ]:
provider "aws" {
  region = var.region
}

# ==================== IAM Roles and Policies ====================

# define a role, which will be assumed by CodeDeploy service
resource "aws_iam_role" "codedeploy_role" {
  name = "appexample-dev-codedeploy-role-us-east-1"
  assume_role_policy = jsonencode({
    Version: "2012-10-17",
    Statement: [
      {
        "Sid": "",
        "Effect": "Allow",
        "Principal": {
          "Service": "codedeploy.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
      }
    ]
  })
}

resource "aws_iam_role_policy_attachment" "codedeploy_role_policy_attachment" {
  role       = aws_iam_role.codedeploy_role.name
  policy_arn = "arn:aws:iam::aws:policy/service-role/AWSCodeDeployRole"
}

# define a role, which will be assumed by a codedeploy app inside an EC2-instance during a deployment
resource "aws_iam_role" "codedeploy_ec2_role" {
  name = "appexample-dev-EC2-codedeploy-role-us-east-1"
  assume_role_policy = jsonencode({
    Version: "2012-10-17",
    Statement: [
      {
        "Sid": "",
        "Effect": "Allow",
        "Principal": {
          "Service": "ec2.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
      }
    ]
  })
}

# define a policy, which allows access to S3 bucket with artifacts
resource "aws_iam_policy" "codedeploy_ec2_to_s3_policy" {
  name        = "appexample-dev-EC2-codedeploy-to-S3-policy-us-east-1"
  description = "Allow access to S3 bucket with artifacts"
  policy = jsonencode({
    Version = "2012-10-17"
    Statement = [
      {
        Effect = "Allow"
        Action = ["s3:Get*", "s3:List*"]
        Resource = "*"
      }
    ]
  })
}

resource "aws_iam_role_policy_attachment" "attach_codedeploy_ec2_to_s3_policy" {
  role       = aws_iam_role.codedeploy_ec2_role.name
  policy_arn = aws_iam_policy.codedeploy_ec2_to_s3_policy.arn
}

# attach the policy to the role
resource "aws_iam_role_policy_attachment" "attach_ssminstancecore_policy" {
  role       = aws_iam_role.codedeploy_ec2_role.name
  policy_arn = "arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore"
}

# define an instance profile, which will be used by an EC2-instance
resource "aws_iam_instance_profile" "attach" {
  name = "appexample-dev-EC2-codedeploy-instanceprofile-us-east-1"
  role = aws_iam_role.codedeploy_ec2_role.name
}

# ==================== S3-Buckets ====================

# define a bucket for storing build artifacts
resource "aws_s3_bucket" "build_artifacts_bucket" {
  bucket = "iic2173-back-terraform"
  force_destroy = true
}

resource "aws_s3_bucket_versioning" "build_artifacts_bucket_versioning" {
  bucket = aws_s3_bucket.build_artifacts_bucket.id
  versioning_configuration {
    status = "Disabled"
  }
}

# ==================== Compute Resources ====================

resource "aws_security_group" "application_sg" {
  name        = "application-sg"
  description = "application-sg"
  ingress {
    from_port   = 22
    to_port     = 22
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }

  ingress {
    from_port   = 80
    to_port     = 80
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }

  ingress {
    from_port   = 443
    to_port     = 443
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }

  egress {
    from_port   = 0
    to_port     = 0
    protocol    = "-1"
    cidr_blocks = ["0.0.0.0/0"]
  }
}

# define a launch template, which will be used by an autoscaling group
resource "aws_launch_template" "auto_scaling_launch_template_test" {
  name_prefix   = "auto_scaling_launch_template_test"
  iam_instance_profile {
    name = aws_iam_instance_profile.attach.name
  }
  image_id      = var.ami
  instance_type = "t2.micro"
  vpc_security_group_ids = [aws_security_group.application_sg.id]
  key_name = var.key_name

  user_data = filebase64("userdata.sh")
}

# define an autoscaling group
resource "aws_autoscaling_group" "autoscaling_group" {
  name                      = "appexample-dev-autoscaling-group-us-east-1"
  desired_capacity          = 1
  max_size                  = 1
  min_size                  = 1
  health_check_grace_period = 300
  health_check_type         = "EC2"
  force_delete              = true
  availability_zones = ["us-east-1a"]
  launch_template {
    id      = aws_launch_template.auto_scaling_launch_template_test.id
    version = "$Latest"
  }
}

# ==================== Codedeploy ====================

# define a CodeDeploy application
resource "aws_codedeploy_app" "codedeploy_app" {
  name = "iic2173-app-terraform"
  compute_platform = "Server"
}

# define a CodeDeploy deployment group
resource "aws_codedeploy_deployment_group" "deployment_group" {
  app_name              = aws_codedeploy_app.codedeploy_app.name
  deployment_group_name = "group-iic2173-terraform"
  service_role_arn      = aws_iam_role.codedeploy_role.arn
  deployment_config_name = "CodeDeployDefault.AllAtOnce"
  autoscaling_groups = [aws_autoscaling_group.autoscaling_group.name]
  auto_rollback_configuration {
    enabled = true
    events  = ["DEPLOYMENT_FAILURE"]
  }
}

resource "null_resource" "deploy" {
  depends_on = [
    aws_codedeploy_app.codedeploy_app,
    aws_codedeploy_deployment_group.deployment_group,
    aws_s3_bucket.build_artifacts_bucket
  ]

  provisioner "local-exec" {
    command = "bash deploy.sh"
  }
}

# ==================== API Gateway ====================

# Define the autoscaling instances
data "aws_instance" "autoscaling_instance" {
  count = 1
  filter {
    name   = "tag:aws:autoscaling:groupName"
    values = [aws_autoscaling_group.autoscaling_group.name]
  }
}

# Define the API Gateway
resource "aws_api_gateway_rest_api" "api" {
  name        = "api-terraform"
  description = "Example API Gateway"
}

# Define a resource in the API Gateway for /example
resource "aws_api_gateway_resource" "example_resource" {
  rest_api_id = aws_api_gateway_rest_api.api.id
  parent_id   = aws_api_gateway_rest_api.api.root_resource_id
  path_part   = "example"
}

# Define a GET method for the /example resource
resource "aws_api_gateway_method" "example_get" {
  rest_api_id   = aws_api_gateway_rest_api.api.id
  resource_id   = aws_api_gateway_resource.example_resource.id
  http_method   = "GET"
  authorization = "NONE"
}

# Define the integration for the GET method on /example
resource "aws_api_gateway_integration" "example_get_integration" {
  rest_api_id = aws_api_gateway_rest_api.api.id
  resource_id = aws_api_gateway_resource.example_resource.id
  http_method = aws_api_gateway_method.example_get.http_method
  type        = "MOCK"
}

# Define a resource in the API Gateway for /fixtures
resource "aws_api_gateway_resource" "fixtures_resource" {
  rest_api_id = aws_api_gateway_rest_api.api.id
  parent_id   = aws_api_gateway_rest_api.api.root_resource_id
  path_part   = "fixtures"
}

# Define a GET method for the /fixtures resource
resource "aws_api_gateway_method" "fixtures_get" {
  rest_api_id   = aws_api_gateway_rest_api.api.id
  resource_id   = aws_api_gateway_resource.fixtures_resource.id
  http_method   = "GET"
  authorization = "NONE"
}

# Modify the integration for the GET method on /fixtures
resource "aws_api_gateway_integration" "fixtures_get_integration" {
  rest_api_id             = aws_api_gateway_rest_api.api.id
  resource_id             = aws_api_gateway_resource.fixtures_resource.id
  http_method             = aws_api_gateway_method.fixtures_get.http_method
  type                    = "HTTP"
  integration_http_method = "GET"
  uri                     = "http://${data.aws_instance.autoscaling_instance[0].public_dns}/fixtures"
}

# Define a deployment for the API Gateway
resource "aws_api_gateway_deployment" "example_deployment" {
  depends_on = [
    aws_api_gateway_integration.example_get_integration,
    aws_api_gateway_integration.fixtures_get_integration
  ]
  rest_api_id = aws_api_gateway_rest_api.api.id
}

# Define a stage for the API Gateway
resource "aws_api_gateway_stage" "example_stage" {
  deployment_id = aws_api_gateway_deployment.example_deployment.id
  rest_api_id   = aws_api_gateway_rest_api.api.id
  stage_name    = "dev"
}
}

main.tf integra recursos como IAM Roles, S3, Auto Scaling, EC2, entre otros. Es una infraestructura importante para que se puedan hacer los displiegues en AWS.

4. __userdata.sh__

In [ ]:
#!/bin/bash

# Update and install Nginx and Docker
sudo apt-get update -y
sudo apt-get install -y nginx docker.io

# Download docker-compose
sudo curl -L "https://github.com/docker/compose/releases/latest/download/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose

# Start and enable services
sudo systemctl enable nginx
sudo systemctl start nginx
sudo systemctl enable docker
sudo systemctl start docker

# Install the CodeDeploy agent
sudo apt-get install ruby wget -y && \
cd /home/ubuntu && \
wget https://aws-codedeploy-us-east-1.s3.us-east-1.amazonaws.com/latest/install && \
chmod +x ./install && \
sudo ./install auto && \
rm install && \
sudo systemctl enable codedeploy-agent && \
sudo systemctl start codedeploy-agent

# Custom Nginx configuration
cat <<EOF | sudo tee /etc/nginx/sites-available/pre-certbot-api.conf
upstream apiLoadBalancer {
    server 0.0.0.0:8001;
    server 0.0.0.0:8002;
    server 0.0.0.0:8003;
}

server {

    server_name entregasarquicrisinguc.xyz www.entregasarquicrisinguc.xyz;

    location / {
        proxy_pass http://apiLoadBalancer;

        proxy_set_header Host \$http_host;
        proxy_set_header X-Real-IP \$remote_addr;
        proxy_set_header X-Forwarded-For \$proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto \$scheme;
    }

    listen 80;
    listen [::]:80;
}

server {
    server_name entregasarquicrisinguc.xyz www.entregasarquicrisinguc.xyz;

    location / {
        proxy_pass http://apiLoadBalancer;

        proxy_set_header Host \$http_host;
        proxy_set_header X-Real-IP \$remote_addr;
        proxy_set_header X-Forwarded-For \$proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto \$scheme;
    }

    listen 80;
    listen [::]:80;
}
EOF

# Enable the custom Nginx configuration
sudo ln -s /etc/nginx/sites-available/pre-certbot-api.conf /etc/nginx/sites-enabled/
sudo rm /etc/nginx/sites-enabled/default

# Restart Nginx to apply the new configuration
sudo systemctl restart nginx

Este archivo ejecuta contenedores de docker, usa instancias de Nginx y recibe despliegues de AWS CodeDeploy.

5. __variables.tf__

In [ ]:
variable "region" {
  default = "us-east-1"
}

variable "ami" {
  default = "ami-0e2c8caa4b6378d8c"
}

variable "key_name" {
  description = "Nombre de la llave SSH para acceder a la instancia"
  type        = string
  default     = "TerraformBack"
}

Este código define las variables de entrada.

6. __.terraform.lock.hcl__

Genera la consistencia de las implementaciones de la infraestructura.

### Final en backend

Usando los código en consola:

In [ ]:
terraform plan
terraform apply

Genera un plan y se aplican los archivos.

## Bonus: Levantar frontend con IaaC

Al igual que el backend, se utilizo Terraform para el levantamiento de este. Además, la forma para iniciar y terminar el flujo de Terraform es con los mismos códigos utlizados en consola.

1. __Carpeta modules__

Esta carpeta tiene subcarpetas, __cloud-front, s3-cf-policiy y s3-static-website__, los cuales son módulos para reutilizar las tareas necesarias para la configuración con el CloudFront, las políticas de acceso a S3 y también el sitio web estático en S3.

2. __locals.tf__

In [ ]:
locals {
  common_tags = {
    company     = var.company
    project     = "${var.company}-${var.project}"
    environment = var.environment
  }

  naming_prefix = "${var.naming_prefix}-${var.environment}"
}

Este archivo reutiliza variables, mantiene consistencia en los tags y genera una mantención más simple.

3. __main.tf__

In [ ]:
####################################################
# Create S3 Static Website
####################################################
module "s3_website" {
  source        = "./modules/s3-static-website"
  bucket_name   = var.bucket_name_primary
  source_files  = "../dist"
  common_tags   = local.common_tags
  naming_prefix = local.naming_prefix
}

####################################################
# Create AWS Cloudfront distribution
####################################################
module "cloud_front" {
  source        = "./modules/cloud-front"
  s3_bucket_id  = module.s3_website.static_website_id
  common_tags   = local.common_tags
  naming_prefix = local.naming_prefix
}

####################################################
# S3 bucket policy to allow access from cloudfront
####################################################
module "s3_cf_policy_primary" {
  source                      = "./modules/s3-cf-policy"
  bucket_id                   = module.s3_website.static_website_id
  bucket_arn                  = module.s3_website.static_website_arn
  cloudfront_distribution_arn = module.cloud_front.cloudfront_distribution_arn
}

Configura la estructura para el front estático, utilizando __Bucket S3, CloudFront y Políticas de S3-CF__.

Los otros archivos del front tienen los mismos objetivos que los archivos del backend, pero con algunos componentes distintos.

## Resumen del flujo para el uso de Terraform

In [ ]:
terraform init
terraform validate
terraform plan
terraform apply
